#**Álgebra Linear Algorítmica 2021.2**

**Professor: João Vitor de Oliveira Silva**

**Laboratório 2**





*Para realizar uma cópia editável deste esqueleto, você pode clicar em Arquivo > Salvar uma cópia no Drive. Você pode remover as células de texto de enunciado e de avisos*, **mas mantendo as células de texto que marcam o início das questões.**

---
###**Avisos**:


*   **Sua solução deve ser devidamente comentada, usando células de texto e/ou códigos desenvolvidos.**
*   **Como já informado, soluções com plágio serão desconsideradas e receberão grau 0.**

*   **O trabalho pode ser feito individualmente ou em dupla.**

* **O nome do seu arquivo contendo a solução deve ser**
$$\mathtt{nome\_DRE\_lab02B.ipynb}$$ ou $$\mathtt{nome1\_DRE\_nome2\_DRE2\_lab02B.ipynb}$$


In [ ]:
# Bibliotecas que sao necessarias ou podem auxiliar a realizacao desta atividade
import numpy as np
import sympy as sym
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt

In [ ]:
#@title Funções que realizam eliminação Gaussiana (sem subs reversa)

def permutation_check(A_aug, l, c):
  for k in range(l+1, A_aug.shape[0]):
    if (A_aug[k, c] != 0):
      return k
    
  return -1


def gaussian_elimination(A, b = None, show_steps = True, save_iters = False):
  """ Computes Gaussian elimination (row echoleon form), showing step by step. 
  Sympy Matrix, Sympy Matrix -> Sympy Matrix """

  if (b == None):
    A_aug = A.copy()
  else:
    A_aug = A.col_insert(A.shape[1], b)

  if (save_iters):
    A_aug_iters = []
    A_aug_iters.append(A_aug.copy())
  
  if (show_steps):
    print('\033[1mInício\033[0m')
    print('--------------------------------------')
    display(A_aug)
    print('\n')
    print('--------------------------------------')

  el = 0
  for j in range(0, A.shape[1]):
    pivot_in_column = False

    if (A_aug[el,j] == 0):
      if (show_steps):
        print('Atenção: pivô zero em A[{0}, {1}]!'. format(el, j) )
        display(A_aug)
      
      l = permutation_check(A_aug, el, j)
      if ( l !=-1 ):
        A_aug[l, :], A_aug[el, :] = A_aug[el, :], A_aug[l, :]
        if (show_steps):
          print('Permutação L{0} <-> L{1}'.format(l, el))
          display(A_aug)
        pivot_in_column = True
        if (save_iters):
          A_aug_iters.append(A_aug.copy())

    else:
      pivot_in_column = True
    
    if ( pivot_in_column ):
      for i in range(el+1, A_aug.shape[0]):
        c = - A_aug[i, j] / A_aug[el, j] 
        A_aug[i, :] = A_aug[i, :] + c * A_aug[el, :]
        if (show_steps):
          print('L{0} <- L{0} + ({2}) * L{1}'.format(i, el, c))
          display(A_aug)
          print('\n')
        if (save_iters):
          A_aug_iters.append(A_aug.copy())
      
      el = (el + 1) if (el + 1 < A_aug.shape[0] ) else el 
      if (show_steps):  
        print('--------------------------------------')

  if (show_steps):
    print('\033[1mFim\033[0m')
  
  if (save_iters):
    return A_aug_iters
  else:
    return A_aug





## Enunciado do problema




Um novo *food-truck* da cidade está se tornando uma febre entre os moradores. Como o mesmo ainda está no início de suas operações, há apenas uma pessoa para atender as solicitações na cozinha e outra para servir o pedido. Os responsáveis pelo *food-truck* estão preocupados em saber se estão dando conta dos pedidos em um tempo razoável ou se estão pecando muito na demora, e pediram a sua ajuda para avaliar essa situação. 

![](https://cdn.pixabay.com/photo/2018/06/04/01/40/stalls-3452163_960_720.png)



Modelaremos este problema como um sistema linear cujas 
variáveis corresponderão a probabilidade de se encontrar o estabelecimento com $i$ pessoas na fila. Por exemplo, $x_2$ seria a probabilidade de se ter 2 pessoas na fila: uma aguardando seu pedido e a outra esperando para realizar o seu. Você foi capaz de reparar algumas coisas:

  - Em 60 minutos de observação (1 hora), foi possível perceber que 28 clientes foram atendidos. Com isso, lhe parece razoável considerar que a probabilidade de um cliente ser atendido numa janela de 1 minuto é de $p_{s} = \frac{28}{60} = \frac{7}{15}$.
  - Em 60 minutos de observação (1 hora), foi possível perceber que 36 pessoas foram até o estabelecimento. Com isso, lhe parece razoável considerar que a probabilidade de um cliente chegar ao estabelecimento numa janela de 1 minuto é de $p_{c} = \frac{36}{60} = \frac{3}{5}$.
  - Quando um cliente via que havia 3 pessoas ou mais na fila, quase sempre desistia e ia para um outro estabelecimento por receio de ter que esperar muito. Logo, você considera em seu modelo que um cliente que chega com 3 pessoas na fila já irá desistir de realizar um pedido.
  - O tempo gasto por uma pessoa descrevendo seu pedido costumou ser muito menor que o tempo de preparação do pedido. Com isso, modela-se que a pessoa que chega imediatamente já aguarda seu pedido (fica desprezado o tempo em que o cliente descreve seu pedido).

É interessante enxergar esse problema por meio de um grafo:


![](https://drive.google.com/uc?id=108VWhVwUEww9T8QZhT5crk1O8gUZ2sGh)


O peso em cada aresta representa a probabilidade de o cenário migrar de um estado $i$ para outro $j$, em que o $i$-ésimo estado representa quantos clientes há na fila em um dado momento. Alguns exemplos são:

-  o peso da aresta $p_s(1-p_c)$ do estado 1 para o estado 0 representa a **probabilidade de se atender o pedido do único cliente na fila** $\color{red}{\text{e}}$ **a probabilidade de não chegar um novo cliente** durante o período de 1 minuto;
- o peso da aresta $(1-p_c)(1-p_s) + p_cp_s$ do estado 1 para o estado 1 representa a **probabilidade de não se atender o pedido do único cliente na fila** $\color{red}{\text{e}}$ **de não chegar um novo cliente na fila** $\color{blue}{\text{ou}}$ **de se atender o pedido do único cliente na fila** $\color{red}{\text{e}}$ **chegar um novo cliente na fila** durante o período de 1 minuto. Ambas as possibilidades irão manter um mesmo número de clientes no sistema.



---



É possível escrever equações de balanço para cada estado. Considerando que o 

$$(\text{Fluxo que entra no estado }i) = (\text{Fluxo que sai do estado } i)$$

é possível escrever para o estado 1 que

$$p_c x_0 + ( (1-p_c)(1-p_s) + p_cp_s)x_1 + p_s(1-p_c) x_2  = p_s(1-p_c)x_1 +  ( (1-p_c)(1-p_s) + p_cp_s)x_1 + p_c(1-p_s)x_1\hspace{1mm}.$$

Percebendo que a igualdade do lado direito pode ser simplificada, a equação pode ser expressa como

$$\begin{align}p_c x_0 + ( (1-p_c)(1-p_s) + p_cp_s)x_1 + p_s(1-p_c) x_2  &= x_1\hspace{1mm},\\
p_c x_0 + ( -1 + ( (1-p_c)(1-p_s) + p_cp_s)) x_1 + p_s(1-p_c) x_2  &= 0
\end{align}.$$


Escrevendo as equações para cada um dos estados, é possível chegar em um sistema linear. Há, entretanto, um pedido a mais que deve ser feito. Como as incógnitas $x_i$ do problema são probabilidades, a seguinte equação também deve ser satisfeita:

$$\sum_{j=0}^3 x_j = 1 \to x_0 + x_1 + x_2 + x_3 = 1\hspace{1mm}.$$

Para resolver um sistema linear, pode usar a função linsolve da biblioteca Sympy como já vimos:

In [ ]:
#@title

# Criando um exemplo de sistema linear Ax = b
A = sym.Matrix([[1, 1, -1, -1],
                [1, 1, -2, 3],
                [2, 0, -3, 2],
                [0, 2, -3, 2] ])
b = sym.Matrix([2, 0, 0, 0, 2])

# Resolvendo o sistema (sem criacao da matriz aumentada e usando funçao solve)
x = list(sym.linsolve((A,b)))[0]
x

(1, 1, 2/5, -2/5)

In [ ]:
#@title Código do simulador (não deve ser alterado, mas DEVE ser executado)
def simulador(time = 720, servers = 1, ps = 7/15, pc = 3/5, max_clients = 3):
  clients = 0
  total_clients = 0
  simulated_freq = np.zeros(max_clients + 1)
  for t in range(0, time):
    print('##########################################')
    print('############ Minuto {0} ################'.format(t))
    # Chegadas
    r = np.random.rand()
    if(r < pc):
      if(clients>=max_clients):
        print('Novo cliente desistiu por ver {0} pessoas aguardando o pedido.'.format(max_clients))
      else:
        clients+=1
        total_clients +=1
        print('Novo cliente chegou ao food-truck. Total de clientes {0}'.format(clients))
    else:
      print('Não chegaram novos clientes ao food-truck. Total de clientes {0}'.format(clients))
    
    simulated_freq[clients] += 1

    # Atendimentos  
    s = 0
    r = np.random.rand()

    if(clients>0 and r < min(clients, servers) * ps ):
      clients-= 1
      print('Pedido concluido.')
      s+= 1

  simulated_freq /= time
  print('-------------------------------------------------')
  print('-------------------------------------------------')
  print('Total de clientes atendidos: {0}'.format(total_clients))
  print('Frequencia de tempo com 0, 1, 2, ou 3 pessoas na fila (probabilidades):')
  print(simulated_freq)
    
    
      
    
      
        
          






Um simulador da situação acima foi realizado, de modo a comparar a solução via sistema e a via simulação.

## Exercício 1





---



Construa um sistema linear $Ax = b$ que modelo o problema acima. Perceba que nem todas as transições estão com seus pesos apresentados no grafo, você deve antes perceber antes quem estes seriam. Em seguida, calcule a solução deste sistema. 

In [ ]:
# 2 -> 2 = pc*ps + (1 - ps)*(1 - pc)
# 3 -> 2 = ps(1 - pc)
pc = sym.Rational(3,5)
ps = sym.Rational(7,15)
"""
Resolvi as equações para cada vértice e deu o seguinte:
x0*pc - x1*ps*(1 - pc) = 0
x0*pc + x1*(-1 + ((1 - pc)*(1 - ps) + pc*ps)) + x2*ps*(1 - pc) = 0 (já feito pelo enunciado)
x1*pc*(1 - ps) - x2*(pc*(1 - ps) + ps*(1 - pc)) + x3*ps*(1 - pc)
x2*pc*(1 - ps) - x3*ps*(1 - pc)
x0 + x1 + x2 + x3 = 1 (já feito no enunciado)
"""
A = sym.Matrix([[pc, -ps*(1 - pc), 0, 0],
                [pc, (-1 + ((1 - pc)*(1 - ps) + pc*ps)), ps*(1 - pc), 0],
                [0, (1 - ps)*pc, -((1 - ps)*pc + (1 - pc)*ps), ps*(1 - pc)],
                [0, 0, pc*(1 - ps), -ps*(1 - pc)],
                [1,1,1,1]])

b = sym.Matrix([0,0,0,0,1])

x = list(sym.linsolve((A, b)))[0]
x

(686/13151, 2205/13151, 3780/13151, 6480/13151)

Resposta: A primeira interrogação que começa no 2 e vai para o 2 n preciso calcular, pois elas se anulariam quando eu fizesse a igualdade. Porém, a interrogação do 3 para o 2 defini como: ps(1 - pc), já que a única probabilidade de perder um na fila é se alguém sair e ninguém entrar no mesmo minuto. Tendo achado o resultado dessa interrogação, calculei o sistema linear:

x0*pc - x1*ps*(1 - pc) = 0
x0*pc + x1*(-1 + ((1 - pc)*(1 - ps) + pc*ps)) + x2*ps*(1 - pc) = 0
x1*pc*(1 - ps) - x2*(pc*(1 - ps) + ps*(1 - pc)) + x3*ps*(1 - pc)
x2*pc*(1 - ps) - x3*ps*(1 - pc)
x0 + x1 + x2 + x3 = 1

Assim, consegui calcular a matriz A e b, coloquei na função "linsolve" e obtive a seguinte solução:

x0 =  686 / 13151 ~ 0,0521
x1 = 2205 / 13151 ~ 0,1676
x2 = 3780 / 13151 ~ 0,2874
x3 = 6480 / 13151 ~ 0,4927

## Exercício 2



 Verifique se sua solução está próxima do simulador abaixo:

In [ ]:
simulador(servers = 1)

##########################################
############ Minuto 0 ################
Não chegaram novos clientes ao food-truck. Total de clientes 0
##########################################
############ Minuto 1 ################
Novo cliente chegou ao food-truck. Total de clientes 1
Pedido concluido.
##########################################
############ Minuto 2 ################
Não chegaram novos clientes ao food-truck. Total de clientes 0
##########################################
############ Minuto 3 ################
Não chegaram novos clientes ao food-truck. Total de clientes 0
##########################################
############ Minuto 4 ################
Novo cliente chegou ao food-truck. Total de clientes 1
Pedido concluido.
##########################################
############ Minuto 5 ################
Novo cliente chegou ao food-truck. Total de clientes 1
##########################################
############ Minuto 6 ################
Não chegaram novos clientes ao food-

Resposta: Arredondando os números para 4 casas decimais, temos que a média de diferença é de:

(|0,0611 - 0,0521| + |0,175 - 0,1676| + |0,2777 - 0,2874| + |0,4861 - 0,4927|) / 4 = (0,009 + 0,0074 + 0,0097 + 0,0066) / 4 = 0,0327 / 4 = 0,008175

Levando em conta que a diferença é de 0,008175, considero que a minha solução está próxima do simulador.

## Exercício 3



Algumas métricas de teoria de filas são usadas em para a avaliar o desempenho de um sistema. Algumas delas são: 

- Probabilidade do sistema estar ocioso: $x_0$
- Número médio de clientes no sistema: $\overline{N} = \sum_{j=0}^3 j \cdot \text{Prob}\{j\text{ clientes no sistema}\} = 0x_1 + 1x_1 + 2x_2 + 3x_3$.
- Probabilidade de alguém ser recusado no sistema: $\text{Prob}\{3\text{ clientes no sistema}\} \cdot \text{Prob}\{\text{novo cliente chegar}\} = x_3 p_c$.

Mostre todas as métricas acima a partir da resolução do sistema.


In [ ]:
# Calculos (pode criar mais celulas de código)
#Arredondarei para 4 casas decimais a fim de facilitar a vizualização
x0 =  686 / 13151
x1 = 2205 / 13151
x2 = 3780 / 13151
x3 = 6480 / 13151

In [ ]:
#Ociosidade
"{0}%".format(round(x0 * 100, 4))

'5.2163%'

In [ ]:
#Número médio de clientes no sistema
x1 + 2*x2 + 3*x3

2.220743669682914

In [ ]:
#Probabilidade de álguem ser recusado no sistema
"{0}%".format(round(x3*pc * 100, 4))

'29.5643%'

Resposta: As métricas foram mostradas acima, o tempo de ociosidade do sistema parece baixo, o que provavelmente mostra que quase sempre há um cliente na fila. A média de clientes é relativamente alta (maior que 1.5), o que, novamente, pode ser sinal da frequente presença de clientes na fila. Por último, a probabilidade de um cliente ser recusado é alta, possivelmente devido ao fato de que há um limite de 3 clientes na fila.

## Exercício 4



Os responsáveis pelo *food-truck* gostariam de verificar se haveria algum ganho expressivo ao se colocar mais uma pessoa para ser responsável pelo preparo dos pedidos. Você irá assumir que esta nova pessoa terá o mesmo desempenho que a o responsável que já atua no *food-truck*. Por simplicidade, ainda irá assumir que um cliente que chega e se depara com 3 pessoas na fila desistirá do seu pedido.
 A inclusão desta nova pessoa irá agilizar o processo quando houver 2 ou 3 pessoas na fila, uma vez que a probabilidade de um pedido ser atendido será maior. O pedido do cliente 1 ou do cliente 2 poderão ser atendidos por um dos dois responsáveis pelo preparo, diferente de anteriormente em que o cliente 2 necessariamente teria de aguardar o pedido do cliente 1 ser concluído. Em termos matemáticos, tem-se que:

 $$\text{Prob}\{\text{um pedido ser atendido}\} =  \text{Prob}\{\text{um pedido ser atendido por pessoa 1} \color{red}{\text{ ou }} \text{um pedido ser atendido por pessoa 2} \} = p_s + p_s = 2p_s\hspace{1mm}.$$

 Isso faz com que todas as arestas que saem dos estados 2 e 3 tenham $2p_s$ no lugar de $p_s$ em seus pesos.

 

Crie agora um sistema linear que modele esta nova situação e resolva o mesmo.

In [ ]:
# Calculos (pode criar mais celulas de código)
# Substituindo os devidos valores Ps por 2Ps, temos o seguinte sistema linear:
"""
x0*pc - x1*ps*(1 - pc) = 0
x0*pc - x1*(ps*(1 - pc) + pc*(1 - ps)) + x2*2*ps*(1 - pc) = 0
x1*pc*(1 - ps) - x2*(2*ps*(1 - pc) + pc*(1 - 2*ps)) + x3*2*ps*(1 - pc) = 0
x2*pc*(1 - 2*ps) - x3*2*ps*(1 - pc) = 0
x0 + x1 + x2 + x3 = 0
"""

A = sym.Matrix([[pc, -ps*(1 - pc), 0, 0],
                [pc, -(ps*(1 - pc) + pc*(1 - ps)), 2*ps*(1 - pc), 0],
                [0, pc*(1 - ps), -(2*ps*(1 - pc) + pc*(1 - 2*ps)), 2*ps*(1 - pc)],
                [0,0,pc*(1 - 2*ps), -2*ps*(1 - pc)],
                [1,1,1,1]])

b = sym.Matrix([0,0,0,0,1])

x = list(sym.linsolve((A, b)))[0]
x

(1372/9967, 4410/9967, 3780/9967, 405/9967)

Resposta: Usando novamente uma aproximação de 4 casas decimais, temos que:
x0 = 1372 / 9967 ~ 0,1376
x1 = 4410 / 9967 ~ 0,4424
x2 = 3780 / 9967 ~ 0,3792
x3 =  405 / 9967 ~ 0,0406

## Exercício 5



Verifique se sua solução está próxima do simulador abaixo:

In [ ]:
simulador(servers = 2)

##########################################
############ Minuto 0 ################
Novo cliente chegou ao food-truck. Total de clientes 1
##########################################
############ Minuto 1 ################
Não chegaram novos clientes ao food-truck. Total de clientes 1
##########################################
############ Minuto 2 ################
Não chegaram novos clientes ao food-truck. Total de clientes 1
Pedido concluido.
##########################################
############ Minuto 3 ################
Novo cliente chegou ao food-truck. Total de clientes 1
Pedido concluido.
##########################################
############ Minuto 4 ################
Não chegaram novos clientes ao food-truck. Total de clientes 0
##########################################
############ Minuto 5 ################
Novo cliente chegou ao food-truck. Total de clientes 1
##########################################
############ Minuto 6 ################
Não chegaram novos clientes ao food-

Resposta: Arredondando os números para 4 casas decimais, temos que a média de diferença é de:

(|0,1667 - 0,1376| + |0,4847 - 0,4424| + |0,3138 - 0,3792| + |0,0347 - 0,0406|) / 4 = (0,0291 + 0,0423 + 0,0654 + 0,0059) / 4 = 0,1427 / 4 = 0,035675

Levando em conta que a diferença é de 0,035675 e, embora seja um pouco maior que a anterior, ainda considero que a minha solução está próxima do simulador de certa forma.

## Exercício 6



Assim como no caso anterior, calcule as mesmas métricas:

- Probabilidade do sistema estar ocioso: $x_0$
- Número médio de clientes no sistema: $\overline{N} = \sum_{j=0}^3 j \cdot \text{Prob}\{j\text{ clientes no sistema}\} = 0x_1 + 1x_1 + 2x_2 + 3x_3$.
- Probabilidade de alguém ser recusado no sistema: $\text{Prob}\{3\text{ clientes no sistema}\} \cdot \text{Prob}\{\text{novo cliente chegar}\} = x_3 p_c$.

Mostre todas as métricas acima a partir da resolução do sistema. Na sua opinião, houve uma melhora expressiva no funcionamento do *food-truck*? Justifique. 


In [ ]:
# Calculos (pode criar mais celulas de código)
#Arredondarei para 4 casas decimais a fim de facilitar a vizualização
x0 = 1372 / 9967
x1 = 4410 / 9967
x2 = 3780 / 9967
x3 =  405 / 9967

In [ ]:
#Ociosidade
"{0}%".format(round(x0 * 100, 4))

'13.7654%'

In [ ]:
#Número médio de clientes no sistema
x1 + 2*x2 + 3*x3

1.3228654560048159

In [ ]:
#Probabilidade de álguem ser recusado no sistema
"{0}%".format(round(x3*pc * 100, 4))

'2.4380%'

Resposta: Analisando os números, parece que sim. Como há mais pessoas na cozinha e a liberação é mais rápida quando há duas ou três pessoas na fila, eles acabam ficando mais tempo ocioso, justamente pelas pessoas serem liberadas antes. Além disso, como eles liberam as pessoas com maior velocidade, a média de pessoas na fila diminui. Por fim, com mais cozinheiros a chance da fila estar cheia é menor, por isso a probabilidade de um indivíduo ser recusado no sistema diminui drasticamente.